Importing the necessary libraries

In [5]:
import helpers as h
import json
import pandas as pd
import numpy as np
import os
import sqlite3
import sys
from tqdm import tqdm

Connecting to the database

In [6]:
database = os.path.join("data","numbeo_db.sqlite")
conn = sqlite3.connect(database)

Based on the collected data, a table was generated containing a list of countries with their ID. A table containing data on cities was prepared in a similar way.

In [7]:
country_city_df = pd.read_csv(os.path.join('data','countries_cities.csv'))
country_df = pd.DataFrame(country_city_df['country'].unique()).reset_index()
country_df.columns = ['country_id', 'country']
country_df.head()

,country_id,country
0,0,Albania
1,1,Austria
2,2,Belarus
3,3,Belgium
4,4,Bosnia and Herzegovina


In [8]:
country_city =pd.merge(country_city_df, country_df, left_on='country', right_on='country')
city_df = pd.DataFrame(country_city[['country_id','city']]).reset_index()
city_df.columns = ['city_id', 'country_id','city']
city_df.head()


,city_id,country_id,city
0,0,0,Apollonia
1,1,0,Ballsh
2,2,0,Berat
3,3,0,Burrel
4,4,0,Dhermi


Reading data from `numbeo.json`

In [9]:
with open(os.path.join('data','numbeo.json'), 'r') as f:
    city_json = json.load(f)

country_city_dict = {}    
for country in city_json:
    country_city_dict[country] ={}
    for city in city_json[country]:
        df = pd.DataFrame.from_dict(city_json[country][city])
        df.index = df.index.astype(np.int64)
        country_city_dict[country][city] = df

Noticed a problem where the 'Avg_price.' columns were incorrectly filled in. Data need to be cleaned.

In [10]:
country_city_dict['Poland']['Warsaw'].head(10)

,Type,Avg_price,Range_price
0,"Meal, Inexpensive Restaurant",9.22 €,6.91-13.83
1,"Meal for 2 People, Mid-range Restaurant, Three...",46.09 €,32.26-69.13
2,McMeal at McDonalds (or Equivalent Combo Meal),7.37 €,6.91-8.53
3,Domestic Beer (0.5 liter draught),3.69 €,2.30-4.61
4,Imported Beer (0.33 liter bottle),3.46 €,2.77-4.84
5,Cappuccino (regular),3.21 €,1.84-4.61
6,Coke/Pepsi (0.33 liter bottle),1.62 €,1.04-2.77
7,Water (0.33 liter bottle),1.54 €,0.69-2.77
8,Markets,Edit,None
9,"Milk (regular), (1 liter)",0.95 €,0.69-1.15


In [11]:
dict_type =h.separate_types(country_city_dict=country_city_dict, country='Poland',city='Warsaw',item='Rent Per Month')
separate_types_df = dict_type['Rent Per Month'].set_index('Type')
df_t = separate_types_df['Avg_price'].to_frame().T.reset_index()
df_t

Type,index,Apartment(1bedroom)inCityCentre,Apartment(1bedroom)OutsideofCentre,Apartment(3bedrooms)inCityCentre,Apartment(3bedrooms)OutsideofCentre
0,Avg_price,98832,74970,172884,126594


Creating `categories` table

In [12]:
df = country_city_dict['Poland']['Warsaw']
categories = df.loc[df['Avg_price']=='Edit'].Type.to_frame(name = 'name')
categories.reset_index(inplace=True)
categories.columns = ['category_id', 'name']

Preparing dataframe for selected categories: Rent Per Month, Buy Apartment Price, Salaries And Financing


For the sake of further analysis on the cost of renting/buying an apartment, I will only focus on 3 categories. 

In [13]:
avg_rent_df= pd.DataFrame()
avg_buy_apartment_df= pd.DataFrame()
avg_salaries_df = pd.DataFrame()
item_names = ['Rent Per Month', 'Buy Apartment Price','Salaries And Financing']

for country in country_city_dict.keys():
    for city in country_city_dict[country].keys():
        avg_rent_df= pd.concat([h.prepare_df(country_city_dict=country_city_dict, 
                            country_city_df=country_city_df,
                            categories=categories,
                            country=country,
                            city=city,
                            item=item_names[0]
                            ), avg_rent_df])

        avg_buy_apartment_df= pd.concat([h.prepare_df(country_city_dict=country_city_dict, 
                            country_city_df=country_city_df,
                            categories=categories,
                            country=country,
                            city=city,
                            item=item_names[1]
                            ), avg_buy_apartment_df])

        avg_salaries_df= pd.concat([h.prepare_df(country_city_dict=country_city_dict, 
                            country_city_df=country_city_df,
                            categories=categories,
                            country=country,
                            city=city,
                            item=item_names[2]
                            ), avg_salaries_df])
    

In [14]:
avg_rent_df.columns = ['category_id', 'apartment_1_bedroom_in_city_centre',
       'apartment_1_bedroom_outside_of_centre',
       'apartment_3bedrooms_in_city_centre',
       'apartment_3bedrooms_outside_of_centre']

avg_buy_apartment_df.columns = ['category_id', 'price_per_square_meter_in_city_centre',
       'price_per_square_meter_outside_of_centre']

avg_salaries_df.columns = ['category_id', 'average_monthly_net_salary',
       'mortgage_interest_rate_in_percentages']

In [15]:
idx_cracow = city_df.loc[city_df['city']=='Krakow (Cracow)'].index.item()
city_df.at[idx_cracow, 'city']='Krakow'

Saving dataframe to database

In [16]:
categories.to_sql(name='categories', con=conn, if_exists='replace')
country_df.to_sql(name='countries', con=conn, if_exists='replace')
city_df.to_sql(name='cities', con=conn, if_exists='replace')
avg_rent_df.to_sql(name='rents', con=conn, if_exists='replace')
avg_buy_apartment_df.to_sql(name='buy_apartments', con=conn, if_exists='replace')
avg_salaries_df.to_sql(name='salaries', con=conn, if_exists='replace')

2346